## Senate NLP Project - Data Clean & Prep
### By: Mitch Brinkman

In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import patsy
import re
import pickle
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer 
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.probability import FreqDist, ConditionalFreqDist
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
import sys
sys.setrecursionlimit(20000)
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text
pd.set_option('display.max_rows', 500)
from senate_func import descr_table
from senate_func import spkr_map_table
from senate_func import speech_table
from senate_func import speech_label
from senate_func import nums
from senate_func import prez
from senate_func import drop_columns

### Table Construction

In [13]:
des_df = descr_table(nums)
des_df.head()

,speech_id,chamber,date,number_within_file,speaker,first_name,last_name,state,gender,line_start,line_end,file,char_count,word_count
0,970000001,S,19810105,1,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,17,62,01051981.txt,1543,283
1,970000002,S,19810105,2,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,65,74,01051981.txt,334,55
2,970000003,S,19810105,3,The VICE,Unknown,Unknown,Unknown,Special,200,212,01051981.txt,466,82
3,970000004,S,19810105,4,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,858,864,01051981.txt,210,37
4,970000005,S,19810105,5,Mr. PERCY,Unknown,PERCY,Unknown,M,883,889,01051981.txt,201,34


In [14]:
spmp_df = spkr_map_table(nums)
spmp_df.head()

,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting
0,97105301,970000005,PERCY,CHARLES,S,IL,M,R,NaN,voting
1,97105071,970000006,HUDDLESTON,WALTER,S,KY,M,D,NaN,voting
2,97106761,970000007,JACKSON,HENRY,S,WA,M,D,NaN,voting
3,97106981,970000008,STAFFORD,ROBERT,S,VT,M,R,NaN,voting
4,97104811,970000010,BAKER,HOWARD,S,TN,M,R,NaN,voting


In [107]:
speech_df = speech_table(nums)
speech_df.head()

Skipping line 1430: '|' expected after '"'
Skipping line 7562: field larger than field limit (131072)
Skipping line 31475: field larger than field limit (131072)
Skipping line 31477: field larger than field limit (131072)
Skipping line 32197: '|' expected after '"'
Skipping line 32273: '|' expected after '"'
Skipping line 32341: '|' expected after '"'
Skipping line 32913: '|' expected after '"'
Skipping line 32969: '|' expected after '"'
Skipping line 33217: '|' expected after '"'
Skipping line 36344: '|' expected after '"'
Skipping line 37641: '|' expected after '"'
Skipping line 47244: '|' expected after '"'
Skipping line 49653: '|' expected after '"'
Skipping line 50739: '|' expected after '"'
Skipping line 67681: '|' expected after '"'
Skipping line 67892: '|' expected after '"'
Skipping line 74817: field larger than field limit (131072)
Skipping line 86258: '|' expected after '"'
Skipping line 87500: '|' expected after '"'
Skipping line 91549: '|' expected after '"'
Skipping line 

Skipping line 195894: '|' expected after '"'
Skipping line 201225: '|' expected after '"'
Skipping line 208703: '|' expected after '"'
Skipping line 212566: '|' expected after '"'
Skipping line 223207: '|' expected after '"'
Skipping line 226311: '|' expected after '"'
Skipping line 228165: '|' expected after '"'
Skipping line 229721: field larger than field limit (131072)
Skipping line 236961: '|' expected after '"'
Skipping line 238582: '|' expected after '"'
Skipping line 244124: '|' expected after '"'
Skipping line 249700: '|' expected after '"'
Skipping line 251043: '|' expected after '"'
Skipping line 251305: '|' expected after '"'
Skipping line 259827: '|' expected after '"'
Skipping line 261116: '|' expected after '"'
Skipping line 262680: '|' expected after '"'
Skipping line 264906: '|' expected after '"'
Skipping line 270900: '|' expected after '"'
Skipping line 279107: '|' expected after '"'
Skipping line 279787: '|' expected after '"'
Skipping line 130773: Expected 2 fields

Skipping line 93248: '|' expected after '"'
Skipping line 94853: '|' expected after '"'
Skipping line 97878: '|' expected after '"'
Skipping line 103569: '|' expected after '"'
Skipping line 106384: '|' expected after '"'
Skipping line 109056: '|' expected after '"'
Skipping line 112837: '|' expected after '"'
Skipping line 113074: '|' expected after '"'
Skipping line 119490: '|' expected after '"'
Skipping line 121437: '|' expected after '"'
Skipping line 122164: '|' expected after '"'
Skipping line 122352: field larger than field limit (131072)
Skipping line 123695: '|' expected after '"'
Skipping line 134534: '|' expected after '"'
Skipping line 140015: '|' expected after '"'
Skipping line 148917: '|' expected after '"'
Skipping line 157007: '|' expected after '"'
Skipping line 158758: '|' expected after '"'
Skipping line 161555: '|' expected after '"'
Skipping line 163740: '|' expected after '"'
Skipping line 165204: '|' expected after '"'
Skipping line 170521: '|' expected after '

Skipping line 205879: '|' expected after '"'
Skipping line 211734: field larger than field limit (131072)
Skipping line 213250: '|' expected after '"'
Skipping line 226465: '|' expected after '"'
Skipping line 229474: '|' expected after '"'
Skipping line 235219: '|' expected after '"'
Skipping line 235543: '|' expected after '"'
Skipping line 237930: '|' expected after '"'
Skipping line 240030: field larger than field limit (131072)
Skipping line 220: Expected 2 fields in line 220, saw 4
Skipping line 11997: Expected 2 fields in line 11997, saw 3
Skipping line 14022: Expected 2 fields in line 14022, saw 4
Skipping line 40818: Expected 2 fields in line 40818, saw 3
Skipping line 40877: Expected 2 fields in line 40877, saw 3
Skipping line 65427: Expected 2 fields in line 65427, saw 3
Skipping line 71789: Expected 2 fields in line 71789, saw 3
Skipping line 75844: Expected 2 fields in line 75844, saw 3
Skipping line 76451: Expected 2 fields in line 76451, saw 4
Skipping line 80795: Expect

Skipping line 74414: '|' expected after '"'
Skipping line 82741: '|' expected after '"'
Skipping line 87089: '|' expected after '"'
Skipping line 90016: '|' expected after '"'
Skipping line 90779: '|' expected after '"'
Skipping line 91245: '|' expected after '"'
Skipping line 91652: '|' expected after '"'
Skipping line 91829: '|' expected after '"'
Skipping line 91837: '|' expected after '"'
Skipping line 99797: '|' expected after '"'
Skipping line 103093: '|' expected after '"'
Skipping line 105537: '|' expected after '"'
Skipping line 114636: '|' expected after '"'
Skipping line 114919: '|' expected after '"'
Skipping line 116581: '|' expected after '"'
Skipping line 119028: '|' expected after '"'
Skipping line 125385: '|' expected after '"'
Skipping line 125387: '|' expected after '"'
Skipping line 125388: '|' expected after '"'
Skipping line 125389: '|' expected after '"'
Skipping line 125390: '|' expected after '"'
Skipping line 130891: '|' expected after '"'
Skipping line 134379

Skipping line 160208: '|' expected after '"'
Skipping line 160209: '|' expected after '"'
Skipping line 160212: '|' expected after '"'
Skipping line 160475: '|' expected after '"'
Skipping line 168530: '|' expected after '"'
Skipping line 171375: '|' expected after '"'
Skipping line 172417: '|' expected after '"'
Skipping line 46412: Expected 2 fields in line 46412, saw 3
Skipping line 47515: Expected 2 fields in line 47515, saw 3
Skipping line 600: '|' expected after '"'
Skipping line 2829: '|' expected after '"'
Skipping line 6183: '|' expected after '"'
Skipping line 12173: '|' expected after '"'
Skipping line 12896: '|' expected after '"'
Skipping line 20340: '|' expected after '"'
Skipping line 20504: '|' expected after '"'
Skipping line 22741: '|' expected after '"'
Skipping line 35297: '|' expected after '"'
Skipping line 36451: '|' expected after '"'
Skipping line 38021: '|' expected after '"'
Skipping line 38353: '|' expected after '"'
Skipping line 44237: '|' expected after '

Skipping line 168222: '|' expected after '"'
Skipping line 171531: '|' expected after '"'
Skipping line 172882: '|' expected after '"'
Skipping line 178517: '|' expected after '"'
Skipping line 179810: '|' expected after '"'
Skipping line 179963: '|' expected after '"'
Skipping line 179975: '|' expected after '"'
Skipping line 180115: '|' expected after '"'
Skipping line 180774: '|' expected after '"'
Skipping line 180775: '|' expected after '"'
Skipping line 180776: '|' expected after '"'
Skipping line 180777: '|' expected after '"'
Skipping line 180778: '|' expected after '"'
Skipping line 180779: '|' expected after '"'
Skipping line 180780: '|' expected after '"'
Skipping line 180781: '|' expected after '"'
Skipping line 180783: '|' expected after '"'
Skipping line 182473: '|' expected after '"'
Skipping line 191969: '|' expected after '"'
Skipping line 193832: '|' expected after '"'
Skipping line 194460: '|' expected after '"'
Skipping line 195748: '|' expected after '"'
Skipping l

In [166]:
new_df = speech_df.merge(spmp_df, on='speech_id')

In [169]:
final_df = new_df.merge(des_df, on=['speech_id','chamber'])
final_df

,speech_id,speech,speakerid,lastname,firstname,chamber,state_x,gender_x,party,district,...,speaker,first_name,last_name,state_y,gender_y,line_start,line_end,file,char_count,word_count
0,970000005,respectively. advanced to the desk of the Vive...,97105301,PERCY,CHARLES,S,IL,M,R,NaN,...,Mr. PERCY,Unknown,PERCY,Unknown,M,883,889,01051981.txt,201,34
1,970000006,respectively. advanced to the desk of the Vice...,97105071,HUDDLESTON,WALTER,S,KY,M,D,NaN,...,Mr. HUDDLESTON,Unknown,HUDDLESTON,Unknown,M,895,901,01051981.txt,202,34
2,970000007,respectively. advanced to the desk of the Vice...,97106761,JACKSON,HENRY,S,WA,M,D,NaN,...,Mr. JAcKSON,Unknown,JACKSON,Unknown,M,907,913,01051981.txt,201,34
3,970000008,respectively. advanced to the desk of the Vice...,97106981,STAFFORD,ROBERT,S,VT,M,R,NaN,...,Mr. STAFFORD,Unknown,STAFFORD,Unknown,M,932,938,01051981.txt,201,34
4,970000010,Mr. President. I suggest the absence of a quorum.,97104811,BAKER,HOWARD,S,TN,M,R,NaN,...,Mr. BAKER,Unknown,BAKER,Unknown,M,986,987,01051981.txt,49,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569070,1140112244,Mr. Speaker. I rise today to congratulate the ...,114118980,COFFMAN,MIKE,H,CO,M,R,6.0,...,Mr. COFFMAN,Unknown,COFFMAN,Unknown,M,7030,7050,09092016.txt,682,109
2569071,1140112245,Mr. Speaker. I rise today because this week is...,114122790,DOLD,BOB,H,IL,M,R,10.0,...,Mr. DOLD,Unknown,DOLD,Unknown,M,7074,7105,09092016.txt,1041,179
2569072,1140112246,Mr. Speaker. it has been 2 years since the peo...,114122890,LIEU,TED,H,CA,M,D,33.0,...,Mr. TED LIEU of California,TED,LIEU,California,M,7110,7141,09092016.txt,1058,189
2569073,1140112247,Mr. Speaker. the University of Southern Indian...,114118760,BUCSHON,LARRY,H,IN,M,R,8.0,...,Mr. BUCSHON,Unknown,BUCSHON,Unknown,M,7152,7179,09092016.txt,868,125


In [174]:
final_clean_df = final_df.loc[(final_df['chamber']=='S') & (final_df['word_count']>=500)]
final_clean_df

,speech_id,speech,speakerid,lastname,firstname,chamber,state_x,gender_x,party,district,...,speaker,first_name,last_name,state_y,gender_y,line_start,line_end,file,char_count,word_count
107,970000182,Mr. President. getting this countrys economy m...,97106241,MATHIAS,CHARLES,S,MD,M,R,NaN,...,Mr. MATHIAS,Unknown,MATHIAS,Unknown,M,3780,3985,01051981.txt,7255,1254
108,970000183,Mr. President. this bill has a simple purpose....,97109651,PRYOR,DAVID,S,AR,M,D,NaN,...,Mr. PRYOR,Unknown,PRYOR,Unknown,M,4120,4276,01051981.txt,5347,824
109,970000184,Mr. Pres.ident. the legislation I introduce to...,97111021,CHAFEE,JOHN,S,RI,M,R,NaN,...,Mr. CHAFEE,Unknown,CHAFEE,Unknown,M,4521,4711,01051981.txt,6957,1131
110,970000185,Mr. President. I am pleased to join my colleag...,97108781,BRADLEY,WILLIAM,S,NJ,M,D,NaN,...,Mr. BRADLEY,Unknown,BRADLEY,Unknown,M,4712,4807,01051981.txt,3119,508
111,970000186,Mr. President. during the past year a very ser...,97113921,WARNER,JOHN,S,VA,M,R,NaN,...,Mr. WARNER,Unknown,WARNER,Unknown,M,4815,5007,01051981.txt,6652,1106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568905,1140111981,Mr. President. I have spoken many times on thi...,114118361,MCCONNELL,MITCH,S,KY,M,R,NaN,...,Mr. MCCONNELL,Unknown,MCCONNELL,Unknown,M,26328,26458,09082016.txt,4564,730
2568907,1140111983,Mr. President. this Sunday we will solemnly ob...,114118121,CARDIN,BENJAMIN,S,MD,M,D,NaN,...,Mr. CARDIN,Unknown,CARDIN,Unknown,M,26766,26912,09082016.txt,5170,889
2568909,1140111985,Mr. President. I submit to the Senate the budg...,114118331,ENZI,MICHAEL,S,WY,M,R,NaN,...,Mr. ENZI,Unknown,ENZI,Unknown,M,26931,27104,09082016.txt,5522,901
2568922,1140111999,Mr. President. I come to the floor againI beli...,114121131,RUBIO,MARCO,S,FL,M,R,NaN,...,Mr. RUBIO,Unknown,RUBIO,Unknown,M,30783,31191,09082016.txt,13262,2448


In [175]:
drop_columns(['number_within_file','first_name', 'last_name','state_y', 'gender_y',
             'line_start','line_end', 'file', 'char_count', 'district', 'nonvoting'],final_clean_df)

/opt/anaconda3/envs/metis/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [300]:
final_clean_df.rename(columns= {'state_x': 'state','gender_x':'gender'}, inplace=True)

In [177]:
final_clean_df['date'] = pd.to_datetime(final_clean_df['date'], format='%Y%m%d')

/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [179]:
pickle.dump(final_clean_df, open("./data/pickles/congress_df.pkl", "wb"))
# final_clean_df.to_pickle('congress_df.pkl')

### Labeling of Speeches

In [318]:
congress_df = pd.read_pickle('./data/pickles/congress_df.pkl')

In [ ]:
edu_df = label_congress_df.loc[(label_congress_df['labels']=='Education')]

In [327]:
congress_df['labels'] = congress_df.speech.apply(lambda x: speech_label(x))

In [319]:
congress_df.reset_index(drop=True, inplace=True)

In [329]:
pickle.dump(congress_df, open("./data/pickles/congress_label_df.pkl", "wb"))

### Segmentation Tables

#### senate table prep before slice & dice

In [ ]:
congress_df = pd.read_pickle('./data/pickles/congress_label_df.pkl')

In [344]:
label_congress_df = congress_df[congress_df.labels != 'None']
label_congress_df.reset_index(drop=True, inplace=True)

In [346]:
pickle.dump(label_congress_df, open("./data/pickles/trim_label_congress_df.pkl", "wb"))

#### Topic/Gender/Party/Region Splits

##### DF Types & Names:
    Topic
        Education: edu_df
        Healthcare: hc_df
        Banking: fin_df
    Era
        1980-1988: reagan_df
        1989-1992: bush_df
        1993-2000: clinton_df
        2001-2008: w_bush_df
        2009-2016: obama_df
    Gender
        female_df
        male_df
        edu_female_df
        hc_female_df
        fin_female_df
        edu_male_df
        hc_male_df
        fin_male_df 
    Party
        edu_dem_df
        hc_dem_df
        fin_dem_df
        edu_rep_df
        hc_rep_df
        fin_rep_df
    

##### Main Theme Tables

In [330]:
edu_df = label_congress_df.loc[(label_congress_df['labels']=='Education')]
hc_df = label_congress_df.loc[(label_congress_df['labels']=='Healthcare')]
fin_df = label_congress_df.loc[(label_congress_df['labels']=='Financial')]

##### By Administration Tables

In [349]:
# splitting up the main database into time frames fitting each administration since 1980
reagan_df = label_congress_df[(label_congress_df['date'] < '1989-01-01')]
bush_df = label_congress_df[(label_congress_df['date'] > '1989-01-01') & (label_congress_df['date'] < '1993-01-01')]
clinton_df = label_congress_df[(label_congress_df['date'] > '1993-01-01') & (label_congress_df['date'] < '2001-01-01')]
w_bush_df = label_congress_df[(label_congress_df['date'] > '2001-01-01') & (label_congress_df['date'] < '2009-01-01')]
obama_df = label_congress_df[(label_congress_df['date'] > '2009-01-01') & (label_congress_df['date'] < '2016-12-01')]

##### Theme-Gender & Theme-Party Tables

In [404]:
female_df = label_congress_df.loc[(label_congress_df['gender']=='F')]
male_df = label_congress_df.loc[(label_congress_df['gender']=='M')]
edu_female_df = edu_df.loc[(edu_df['gender']=='F')]
edu_male_df = edu_df.loc[(edu_df['gender']=='M')]
hc_female_df = hc_df.loc[(hc_df['gender']=='F')]
hc_male_df = hc_df.loc[(hc_df['gender']=='M')]
fin_female_df = fin_df.loc[(fin_df['gender']=='F')]
fin_male_df = fin_df.loc[(fin_df['gender']=='M')]
hc_dem_df = hc_df.loc[(hc_df['party']=='D')]
hc_rep_df = hc_df.loc[(hc_df['party']=='R')]
edu_dem_df = edu_df.loc[(edu_df['party']=='D')]
edu_rep_df = edu_df.loc[(edu_df['party']=='R')]
fin_dem_df = fin_df.loc[(fin_df['party']=='D')]
fin_rep_df = fin_df.loc[(fin_df['party']=='R')]

In [ ]:
## pickle code to run for each DF
pickle.dump(female_df, open("./data/pickles/gender/female_df.pkl", "wb"))

### Code Archive - DISREGARD

In [326]:
# def sentiment_speech_label(text):
# #     text = text.lower()
# #     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     speech_length = len(text)
#     token_text = TreebankWordTokenizer().tokenize(text)
#     fdist = FreqDist(token_text)
#     hc_freq = (fdist['healthcare'] / speech_length) +(fdist['health'] / speech_length) +(fdist['care'] / speech_length)+(fdist['medicare'] / speech_length) + (fdist['medicaid'] / speech_length) + (fdist['medicine'] / speech_length)
#     edu_freq = (fdist['education'] / speech_length) + (fdist['school'] / speech_length) +(fdist['students'] / speech_length)
#     fin_freq = (fdist['bankers'] / speech_length) + (fdist['banking'] / speech_length)+(fdist['banks'] / speech_length)+(fdist['loans'] / speech_length)
#     if fin_freq > 0.003 and fin_freq > edu_freq and fin_freq > hc_freq:
#             return 'Financial'
#     elif hc_freq > 0.004 and hc_freq > fin_freq and hc_freq > edu_freq:
#             return 'Healthcare'
#     elif edu_freq > 0.004 and edu_freq > fin_freq and edu_freq > hc_freq:
#             return 'Education'
#     else:
#         return 'None'

In [326]:
# def speech_label(text):
#     text = text.lower()
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     speech_length = len(text)
#     token_text = TreebankWordTokenizer().tokenize(text)
#     fdist = FreqDist(token_text)
#     hc_freq = (fdist['healthcare'] / speech_length) +(fdist['health'] / speech_length) +(fdist['care'] / speech_length)+(fdist['medicare'] / speech_length) + (fdist['medicaid'] / speech_length) + (fdist['medicine'] / speech_length)
#     edu_freq = (fdist['education'] / speech_length) + (fdist['school'] / speech_length) +(fdist['students'] / speech_length)
#     fin_freq = (fdist['bankers'] / speech_length) + (fdist['banking'] / speech_length)+(fdist['banks'] / speech_length)+(fdist['loans'] / speech_length)
#     if fin_freq > 0.003 and fin_freq > edu_freq and fin_freq > hc_freq:
#             return 'Financial'
#     elif hc_freq > 0.004 and hc_freq > fin_freq and hc_freq > edu_freq:
#             return 'Healthcare'
#     elif edu_freq > 0.004 and edu_freq > fin_freq and edu_freq > hc_freq:
#             return 'Education'
#     else:
#         return 'None'